<a href="https://colab.research.google.com/github/tanishkabishtt/customer-data-cleaning/blob/main/data_cleaning_customer_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving customer_raw_data.csv to customer_raw_data.csv


In [ ]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv("customer_raw_data.csv", dtype=str)  # read all as string first


In [ ]:
# Strip whitespace from all string cells
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Standardize case
df["Name"] = df["Name"].str.title()
df["City"] = df["City"].str.strip("-").str.strip().str.title()  # handle "-Unknown-"


/tmp/ipython-input-1835355031.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [ ]:
# Ensure numeric and drop rows without valid IDs
df["CustomerID"] = pd.to_numeric(df["CustomerID"], errors="coerce")
df = df.dropna(subset=["CustomerID"])
df["CustomerID"] = df["CustomerID"].astype(int)


In [ ]:
# Convert to numeric
df["Age"] = pd.to_numeric(df["Age"], errors="coerce")

# Invalidate zeros and extremes
df.loc[(df["Age"].isna()) | (df["Age"] < 15) | (df["Age"] > 100), "Age"] = np.nan


In [ ]:
# Normalize blank and unknown
df["City"] = df["City"].replace({"": np.nan, "Unknown": np.nan})

# Optional: map common variants
city_map = {
    "Delhi": "Delhi",
    "Mumbai": "Mumbai",
    "Pune": "Pune",
    "Chennai": "Chennai",
    "Kolkata": "Kolkata",
    "Jaipur": "Jaipur",
}
df["City"] = df["City"].map(lambda x: city_map.get(x, x) if pd.notna(x) else x)


In [ ]:
# Remove currency symbols and non-digits; set missing/invalid to NaN
def clean_amount(x):
    if pd.isna(x):
        return np.nan
    x = str(x).lower().strip()
    if x in ["", "na", "n/a", "none", "null", "not available", "?"]:
        return np.nan
    # Keep digits only
    digits = re.sub(r"[^\d]", "", x)
    return float(digits) if digits else np.nan

df["PurchaseAmount"] = df["PurchaseAmount"].apply(clean_amount)


In [ ]:
# Try multiple date formats; coerce invalids to NaN
df["JoinDate"] = pd.to_datetime(
    df["JoinDate"],
    errors="coerce",
    infer_datetime_format=True,
    dayfirst=False  # set True if needed for your locale
)

# Optional: remove future dates if you want historic-only
from datetime import datetime
today = pd.Timestamp.today()
df.loc[(df["JoinDate"] > today) | (df["JoinDate"].isna()), "JoinDate"] = df["JoinDate"]  # keep NaN for invalids


/tmp/ipython-input-3297572552.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["JoinDate"] = pd.to_datetime(


In [ ]:
def clean_phone(x):
    if pd.isna(x):
        return np.nan
    digits = re.sub(r"\D", "", str(x))
    # Keep exactly 10 digits
    if len(digits) != 10:
        return np.nan
    # Reject all identical digits (e.g., 0000000000, 9999999999)
    if len(set(digits)) == 1:
        return np.nan
    return digits

df["Phone"] = df["Phone"].apply(clean_phone)


In [ ]:
# If two rows have same CustomerID, keep the first non-null fields (simple rule)
df = df.sort_values(["CustomerID", "JoinDate"], na_position="last")
df = df.drop_duplicates(subset=["CustomerID"], keep="first")


In [ ]:
df["non_nulls"] = df.notna().sum(axis=1)
df = df.sort_values(["CustomerID", "non_nulls"], ascending=[True, False]).drop_duplicates("CustomerID")
df = df.drop(columns=["non_nulls"])


In [ ]:
df = df.astype({
    "CustomerID": "int64",
    "Name": "string",
    "Age": "Int64",           # nullable integer
    "City": "string",
    "PurchaseAmount": "float",
    "JoinDate": "datetime64[ns]",
    "Phone": "string"
})

df.to_csv("customer_cleaned.csv", index=False)


In [ ]:
print(df.head())


   CustomerID     Name   Age    City  PurchaseAmount   JoinDate       Phone
0         101     Riya    21   Delhi          2000.0 2020-05-12  9876543210
1         102  Rahul@@  <NA>  Mumbai          3500.0        NaT        <NA>
2         103     <NA>  <NA>    Pune          4500.0        NaT  9876500000
3         104     Amit    25    <NA>             NaN 2020-07-20  9876543210
4         105    Sneha  <NA>   Delhi          5000.0        NaT  9988776655


In [ ]:
df



,CustomerID,Name,Age,City,PurchaseAmount,JoinDate,Phone
0,101,Riya,21,Delhi,2000.0,2020-05-12,9876543210
1,102,Rahul@@,<NA>,Mumbai,3500.0,NaT,<NA>
2,103,<NA>,<NA>,Pune,4500.0,NaT,9876500000
3,104,Amit,25,<NA>,NaN,2020-07-20,9876543210
4,105,Sneha,<NA>,Delhi,5000.0,NaT,9988776655
5,106,Priya,22,Mumbai,NaN,NaT,9988776655
6,107,Rohan###,<NA>,Chennai,1500.0,NaT,<NA>
7,108,Meera,19,Delhi,NaN,NaT,<NA>
8,109,Amit,25,Delhi,4500.0,NaT,<NA>
9,110,Raj,<NA>,Kolkata,3500.0,NaT,<NA>


In [ ]:
df["Name"] = df["Name"].str.replace(r"[^A-Za-z\s]", "", regex=True).str.strip()
df["Name"] = df["Name"].replace({"": pd.NA})


In [ ]:
df["Age"] = pd.to_numeric(df["Age"], errors="coerce")
df.loc[(df["Age"] < 15) | (df["Age"] > 100), "Age"] = pd.NA


In [ ]:
df["City"] = df["City"].str.title().replace({"": pd.NA})


In [ ]:
df["PurchaseAmount"] = pd.to_numeric(df["PurchaseAmount"], errors="coerce")


In [ ]:
df["JoinDate"] = pd.to_datetime(df["JoinDate"], errors="coerce", infer_datetime_format=True)


/tmp/ipython-input-3974231533.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["JoinDate"] = pd.to_datetime(df["JoinDate"], errors="coerce", infer_datetime_format=True)


In [ ]:
import re
def clean_phone(x):
    if pd.isna(x): return pd.NA
    digits = re.sub(r"\D", "", str(x))
    if len(digits) != 10 or len(set(digits)) == 1: return pd.NA
    return digits

df["Phone"] = df["Phone"].apply(clean_phone)


In [ ]:
df = df.drop_duplicates(subset=["CustomerID"])


In [ ]:
print(df.info())
print(df.isna().sum())
print(df["Age"].describe())
print(df["City"].value_counts(dropna=False))
print(df["JoinDate"].dropna().min(), df["JoinDate"].dropna().max())


<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 0 to 16
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   CustomerID      16 non-null     int64         
 1   Name            14 non-null     string        
 2   Age             8 non-null      Int64         
 3   City            13 non-null     string        
 4   PurchaseAmount  13 non-null     float64       
 5   JoinDate        4 non-null      datetime64[ns]
 6   Phone           8 non-null      object        
 7   	JoinDate       4 non-null      datetime64[ns]
dtypes: Int64(1), datetime64[ns](2), float64(1), int64(1), object(1), string(2)
memory usage: 1.1+ KB
None
CustomerID         0
Name               2
Age                8
City               3
PurchaseAmount     3
JoinDate          12
Phone              8
\tJoinDate        12
dtype: int64
count         8.0
mean         21.5
std      2.828427
min          17.0
25%         19.75
50%  

In [ ]:
df

,CustomerID,Name,Age,City,PurchaseAmount,JoinDate,Phone,\tJoinDate
0,101,Riya,21,Delhi,2000.0,2020-05-12,9876543210,2020-05-12
1,102,Rahul,<NA>,Mumbai,3500.0,NaT,<NA>,NaT
2,103,<NA>,<NA>,Pune,4500.0,NaT,9876500000,NaT
3,104,Amit,25,<NA>,NaN,2020-07-20,9876543210,2020-07-20
4,105,Sneha,<NA>,Delhi,5000.0,NaT,9988776655,NaT
5,106,Priya,22,Mumbai,NaN,NaT,9988776655,NaT
6,107,Rohan,<NA>,Chennai,1500.0,NaT,<NA>,NaT
7,108,Meera,19,Delhi,NaN,NaT,<NA>,NaT
8,109,Amit,25,Delhi,4500.0,NaT,<NA>,NaT
9,110,Raj,<NA>,Kolkata,3500.0,NaT,<NA>,NaT


In [ ]:
print(df.columns.tolist())


['CustomerID', 'Name', 'Age', 'City', 'PurchaseAmount', 'JoinDate', 'Phone', '\tJoinDate']


In [ ]:
df = df.drop(columns=["\tJoinDate"])


In [ ]:
print(df.columns)


Index(['CustomerID', 'Name', 'Age', 'City', 'PurchaseAmount', 'JoinDate',
       'Phone'],
      dtype='object')


In [ ]:
df

,CustomerID,Name,Age,City,PurchaseAmount,JoinDate,Phone
0,101,Riya,21,Delhi,2000.0,2020-05-12,9876543210
1,102,Rahul,<NA>,Mumbai,3500.0,NaT,<NA>
2,103,<NA>,<NA>,Pune,4500.0,NaT,9876500000
3,104,Amit,25,<NA>,NaN,2020-07-20,9876543210
4,105,Sneha,<NA>,Delhi,5000.0,NaT,9988776655
5,106,Priya,22,Mumbai,NaN,NaT,9988776655
6,107,Rohan,<NA>,Chennai,1500.0,NaT,<NA>
7,108,Meera,19,Delhi,NaN,NaT,<NA>
8,109,Amit,25,Delhi,4500.0,NaT,<NA>
9,110,Raj,<NA>,Kolkata,3500.0,NaT,<NA>


In [ ]:
report = df.isna().sum().reset_index()
report.columns = ["Column", "MissingCount"]
print(report)


           Column  MissingCount
0      CustomerID             0
1            Name             2
2             Age             8
3            City             3
4  PurchaseAmount             3
5        JoinDate            12
6           Phone             8


In [ ]:
from google.colab import files
files.download("customer_cleaned.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df

,CustomerID,Name,Age,City,PurchaseAmount,JoinDate,Phone
0,101,Riya,21,Delhi,2000.0,2020-05-12,9876543210
1,102,Rahul,<NA>,Mumbai,3500.0,NaT,<NA>
2,103,<NA>,<NA>,Pune,4500.0,NaT,9876500000
3,104,Amit,25,<NA>,NaN,2020-07-20,9876543210
4,105,Sneha,<NA>,Delhi,5000.0,NaT,9988776655
5,106,Priya,22,Mumbai,NaN,NaT,9988776655
6,107,Rohan,<NA>,Chennai,1500.0,NaT,<NA>
7,108,Meera,19,Delhi,NaN,NaT,<NA>
8,109,Amit,25,Delhi,4500.0,NaT,<NA>
9,110,Raj,<NA>,Kolkata,3500.0,NaT,<NA>


In [ ]:
df.to_csv("customer_cleaned.csv", index=False)


In [ ]:
from google.colab import files
files.download("customer_cleaned.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>